
80. トークン化
“The movie was full of incomprehensibilities.”という文をトークンに分解し、トークン列を表示せよ。

In [1]:
from transformers import AutoTokenizer

# 第一步：加载 BERT tokenizer
tokenizer = AutoTokenizer.from_pretrained("bert-base-uncased")

# 输入句子
sentence = "The movie was full of incomprehensibilities."

# 第二步：进行分词
tokens = tokenizer.tokenize(sentence)

# 第三步：输出 token 序列
print(tokens)


/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

['the', 'movie', 'was', 'full', 'of', 'inc', '##omp', '##re', '##hen', '##si', '##bilities', '.']


81. マスクの予測
“The movie was full of [MASK].”の”[MASK]”を埋めるのに最も適切なトークンを求めよ。

In [2]:
import torch
from transformers import AutoTokenizer, BertForMaskedLM

# 第一步：加载 tokenizer とモデル
tokenizer = AutoTokenizer.from_pretrained("bert-base-uncased")
model = BertForMaskedLM.from_pretrained("bert-base-uncased")
model.eval() # モデルを評価モードに設定

# 入力文（[MASK] を使用することに注意）
sentence = "The movie was full of [MASK]."

# 第二歩：エンコード
inputs = tokenizer(sentence, return_tensors="pt")

# [MASK] の位置を見つける
mask_token_index = torch.where(inputs["input_ids"] == tokenizer.mask_token_id)[1]

# モデル予測
with torch.no_grad():
    outputs = model(**inputs)

# [MASK] トークンのロジットを取得
# outputs.logits の形状は (batch_size, sequence_length, vocab_size)
# ここでは batch_size=1 なので、0番目の要素にアクセスし、mask_token_index でマスク位置のロジットを取得
mask_token_logits = outputs.logits[0, mask_token_index, :]

# 最も確率の高いトークンIDを取得
predicted_token_id = torch.argmax(mask_token_logits).item()

# トークンIDを単語に変換
predicted_token = tokenizer.decode(predicted_token_id)

print(f"最も適切なトークン: {predicted_token}")

model.safetensors:   0%|          | 0.00/440M [00:00<?, ?B/s]

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForMaskedLM: ['bert.pooler.dense.bias', 'bert.pooler.dense.weight', 'cls.seq_relationship.bias', 'cls.seq_relationship.weight']
- This IS expected if you are initializing BertForMaskedLM from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForMaskedLM from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


最も適切なトークン: fun


82. マスクのtop-k予測
“The movie was full of [MASK].”の”[MASK]”に埋めるのに適切なトークン上位10個と、その確率（尤度）を求めよ。

In [3]:
import torch
from transformers import AutoTokenizer, BertForMaskedLM

# 第一步：加载 tokenizer 和模型
tokenizer = AutoTokenizer.from_pretrained("bert-base-uncased")
model = BertForMaskedLM.from_pretrained("bert-base-uncased")
model.eval() # モデルを評価モードに設定

# 输入句子
sentence = "The movie was full of [MASK]."

# 编码
inputs = tokenizer(sentence, return_tensors="pt")

# 找到 [MASK] 位置
mask_token_index = torch.where(
    inputs["input_ids"] == tokenizer.mask_token_id
)[1]

# モデル予測
with torch.no_grad():
    outputs = model(**inputs)

logits = outputs.logits

# 第二步：取 [MASK] 位置の logits 并 softmax
mask_logits = logits[0, mask_token_index, :]
probabilities = torch.softmax(mask_logits, dim=1)

# 第三步：Top-10
top_k = 10
top_k_probs, top_k_ids = torch.topk(probabilities, top_k, dim=1)

# 結果の表示
print("上位10個のトークンとそれらの確率:")
for i in range(top_k):
    token_id = top_k_ids[0][i].item()
    probability = top_k_probs[0][i].item()
    token = tokenizer.decode(token_id)
    print(f"トークン: {token}, 確率: {probability:.4f}")

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForMaskedLM: ['bert.pooler.dense.bias', 'bert.pooler.dense.weight', 'cls.seq_relationship.bias', 'cls.seq_relationship.weight']
- This IS expected if you are initializing BertForMaskedLM from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForMaskedLM from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


上位10個のトークンとそれらの確率:
トークン: fun, 確率: 0.1071
トークン: surprises, 確率: 0.0663
トークン: drama, 確率: 0.0447
トークン: stars, 確率: 0.0272
トークン: laughs, 確率: 0.0254
トークン: action, 確率: 0.0195
トークン: excitement, 確率: 0.0190
トークン: people, 確率: 0.0183
トークン: tension, 確率: 0.0150
トークン: music, 確率: 0.0146


83. CLSトークンによる文ベクトル
以下の文の全ての組み合わせに対して、最終層の[CLS]トークンの埋め込みベクトルを用いてコサイン類似度を求めよ。

“The movie was full of fun.”
“The movie was full of excitement.”
“The movie was full of crap.”
“The movie was full of rubbish.”

In [4]:
import torch
from transformers import AutoTokenizer, BertModel
import torch.nn.functional as F
from itertools import combinations

# 第一步：加载 tokenizer 和 BERT 模型
tokenizer = AutoTokenizer.from_pretrained("bert-base-uncased")
model = BertModel.from_pretrained("bert-base-uncased")

# 输入句子
sentences = [
    "The movie was full of fun.",
    "The movie was full of excitement.",
    "The movie was full of crap.",
    "The movie was full of rubbish."
]

# 第二步：获取 CLS 向量
cls_vectors = []

with torch.no_grad():
    for sent in sentences:
        inputs = tokenizer(sent, return_tensors="pt")
        outputs = model(**inputs)
        cls_vector = outputs.last_hidden_state[:, 0, :]  # [CLS]
        cls_vectors.append(cls_vector)

# 第三步：计算余弦相似度
print("Cosine Similarities (CLS-based):\n")

for (i, vec1), (j, vec2) in combinations(enumerate(cls_vectors), 2):
    similarity = F.cosine_similarity(vec1, vec2).item()
    print(f"Sentence {i+1} & Sentence {j+1}: {similarity:.4f}")


Cosine Similarities (CLS-based):

Sentence 1 & Sentence 2: 0.9881
Sentence 1 & Sentence 3: 0.9558
Sentence 1 & Sentence 4: 0.9475
Sentence 2 & Sentence 3: 0.9541
Sentence 2 & Sentence 4: 0.9487
Sentence 3 & Sentence 4: 0.9807


84. 平均による文ベクトル
以下の文の全ての組み合わせに対して、最終層の埋め込みベクトルの平均を用いてコサイン類似度を求めよ。

“The movie was full of fun.”
“The movie was full of excitement.”
“The movie was full of crap.”
“The movie was full of rubbish.”

In [5]:
import torch
from transformers import AutoTokenizer, BertModel
import torch.nn.functional as F
from itertools import combinations

# 第一步：加载 tokenizer 和模型
tokenizer = AutoTokenizer.from_pretrained("bert-base-uncased")
model = BertModel.from_pretrained("bert-base-uncased")

# 输入句子
sentences = [
    "The movie was full of fun.",
    "The movie was full of excitement.",
    "The movie was full of crap.",
    "The movie was full of rubbish."
]

sentence_vectors = []

with torch.no_grad():
    for sent in sentences:
        inputs = tokenizer(sent, return_tensors="pt", padding=True)
        outputs = model(**inputs)

        # 最终层所有 token 的埋め込み
        token_embeddings = outputs.last_hidden_state  # [1, seq_len, hidden]

        # attention mask
        attention_mask = inputs["attention_mask"].unsqueeze(-1)

        # mean pooling（排除 padding）
        summed = torch.sum(token_embeddings * attention_mask, dim=1)
        counts = torch.sum(attention_mask, dim=1)
        mean_vector = summed / counts

        sentence_vectors.append(mean_vector)

# 第三步：计算余弦相似度
print("Cosine Similarities (Mean Pooling):\n")

for (i, v1), (j, v2) in combinations(enumerate(sentence_vectors), 2):
    sim = F.cosine_similarity(v1, v2).item()
    print(f"Sentence {i+1} & Sentence {j+1}: {sim:.4f}")


Cosine Similarities (Mean Pooling):

Sentence 1 & Sentence 2: 0.9568
Sentence 1 & Sentence 3: 0.8490
Sentence 1 & Sentence 4: 0.8169
Sentence 2 & Sentence 3: 0.8352
Sentence 2 & Sentence 4: 0.7938
Sentence 3 & Sentence 4: 0.9226


85. データセットの準備
General Language Understanding Evaluation (GLUE) ベンチマークで配布されているStanford Sentiment Treebank (SST) から訓練セット（train.tsv）と開発セット（dev.tsv）のテキストと極性ラベルと読み込み、さらに全てのテキストはトークン列に変換せよ。

In [8]:
import pandas as pd
from transformers import AutoTokenizer

# 第一步：加载 tokenizer
tokenizer = AutoTokenizer.from_pretrained("bert-base-uncased")

# 第二步：读取 SST 数据集
train_df = pd.read_csv("train.tsv", sep="\t")
dev_df = pd.read_csv("dev.tsv", sep="\t")

# 提取文本和标签
train_texts = train_df["sentence"].tolist()
train_labels = train_df["label"].tolist()

dev_texts = dev_df["sentence"].tolist()
dev_labels = dev_df["label"].tolist()

print(f"Train size: {len(train_texts)}")
print(f"Dev size: {len(dev_texts)}")

# 第三步：将文本转换为 token 序列
train_encodings = tokenizer(
    train_texts,
    padding=True,
    truncation=True,
    return_tensors="pt"
)

dev_encodings = tokenizer(
    dev_texts,
    padding=True,
    truncation=True,
    return_tensors="pt"
)

# 示例：查看第一个训练样本
print("\nExample tokenization:")
print("Sentence:", train_texts[0])
print("Tokens:", tokenizer.convert_ids_to_tokens(train_encodings["input_ids"][0]))
print("Label:", train_labels[0])


Train size: 36985
Dev size: 872

Example tokenization:
Sentence: hide new secretions from the parental units 
Tokens: ['[CLS]', 'hide', 'new', 'secret', '##ions', 'from', 'the', 'parental', 'units', '[SEP]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]']
Label: 0.0


86. ミニバッチの作成
85で読み込んだ訓練データの一部（例えば冒頭の4事例）に対して、パディングなどの処理を行い、トークン列の長さを揃えてミニバッチを構成せよ。

In [9]:
import torch

# 第一步：取训练数据的前 4 条
batch_texts = train_texts[:4]
batch_labels = train_labels[:4]

# 第二步：padding 后进行 tokenization
batch_encodings = tokenizer(
    batch_texts,
    padding=True,
    truncation=True,
    return_tensors="pt"
)

# 第三步：构成 mini-batch
input_ids = batch_encodings["input_ids"]
attention_mask = batch_encodings["attention_mask"]
labels = torch.tensor(batch_labels)

# 输出检查
print("Mini-batch shapes:")
print("input_ids shape:", input_ids.shape)
print("attention_mask shape:", attention_mask.shape)
print("labels shape:", labels.shape)

print("\nExample (first sentence):")
print("Tokens:", tokenizer.convert_ids_to_tokens(input_ids[0]))
print("Attention mask:", attention_mask[0])
print("Label:", labels[0])


Mini-batch shapes:
input_ids shape: torch.Size([4, 15])
attention_mask shape: torch.Size([4, 15])
labels shape: torch.Size([4])

Example (first sentence):
Tokens: ['[CLS]', 'hide', 'new', 'secret', '##ions', 'from', 'the', 'parental', 'units', '[SEP]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]']
Attention mask: tensor([1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0])
Label: tensor(0.)


87. ファインチューニング
訓練セットを用い、事前学習済みモデルを極性分析タスク向けにファインチューニングせよ。検証セット上でファインチューニングされたモデルの正解率を計測せよ。

In [2]:
import pandas as pd
import torch
from torch.utils.data import Dataset, DataLoader
from transformers import AutoTokenizer, BertForSequenceClassification
from torch.optim import AdamW # AdamW のインポートを修正
from tqdm import tqdm

# ========= 1. 读取数据 =========
train_df = pd.read_csv("train.tsv", sep="\t")
dev_df = pd.read_csv("dev.tsv", sep="\t")

train_texts = train_df["sentence"].tolist()
train_labels = train_df["label"].tolist()

dev_texts = dev_df["sentence"].tolist()
dev_labels = dev_df["label"].tolist()

# ========= 2. Tokenizer =========
tokenizer = AutoTokenizer.from_pretrained("bert-base-uncased")

train_encodings = tokenizer(
    train_texts,
    padding=True,
    truncation=True,
    return_tensors="pt"
)

dev_encodings = tokenizer(
    dev_texts,
    padding=True,
    truncation=True,
    return_tensors="pt"
)

# ========= 3. Dataset =========
class SSTDataset(Dataset):
    def __init__(self, encodings, labels):
        self.encodings = encodings
        self.labels = labels

    def __len__(self):
        return len(self.labels)

    def __getitem__(self, idx):
        item = {k: v[idx] for k, v in self.encodings.items()}
        # ラベルを torch.long 型に変換して、single_label_classification として扱われるようにする
        item["labels"] = torch.tensor(self.labels[idx], dtype=torch.long)
        return item

train_dataset = SSTDataset(train_encodings, train_labels)
dev_dataset = SSTDataset(dev_encodings, dev_labels)

train_loader = DataLoader(train_dataset, batch_size=16, shuffle=True)
dev_loader = DataLoader(dev_dataset, batch_size=32)

# ========= 4. Model =========
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

model = BertForSequenceClassification.from_pretrained(
    "bert-base-uncased",
    num_labels=2
)
model.to(device)

optimizer = AdamW(model.parameters(), lr=2e-5)

# ========= 5. Training =========
num_epochs = 3

for epoch in range(num_epochs):
    model.train()
    total_loss = 0

    for batch in tqdm(train_loader):
        optimizer.zero_grad()
        batch = {k: v.to(device) for k, v in batch.items()}

        outputs = model(**batch)
        loss = outputs.loss

        loss.backward()
        optimizer.step()

        total_loss += loss.item()

    print(f"Epoch {epoch+1}, avg loss = {total_loss / len(train_loader):.4f}")


# ========= 6. Evaluation =========
model.eval()
correct = 0
total = 0

with torch.no_grad():
    for batch in dev_loader:
        batch = {k: v.to(device) for k, v in batch.items()}
        outputs = model(**batch)
        preds = torch.argmax(outputs.logits, dim=1)

        correct += (preds == batch["labels"]).sum().item()
        total += batch["labels"].size(0)

print("Dev accuracy:", correct / total)


/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

...

Epoch 1, avg loss = 0.2010


100%|██████████| 4210/4210 [14:26<00:00,  4.86it/s]


Epoch 2, avg loss = 0.1068


100%|██████████| 4210/4210 [14:26<00:00,  4.86it/s]


Epoch 3, avg loss = 0.0706
Dev accuracy: 0.9128440366972477


88. 極性分析
問題87でファインチューニングされたモデルを用いて、以下の文の極性を予測せよ。

“The movie was full of incomprehensibilities.”
“The movie was full of fun.”
“The movie was full of excitement.”
“The movie was full of crap.”
“The movie was full of rubbish.”


In [3]:
import torch

# 测试句子
test_sentences = [
    "The movie was full of incomprehensibilities.",
    "The movie was full of fun.",
    "The movie was full of excitement.",
    "The movie was full of crap.",
    "The movie was full of rubbish."
]

# 标签映射（便于阅读）
label_map = {0: "Negative", 1: "Positive"}

# Tokenize
inputs = tokenizer(
    test_sentences,
    padding=True,
    truncation=True,
    return_tensors="pt"
)

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
inputs = {k: v.to(device) for k, v in inputs.items()}
model.to(device)

# 推論
model.eval()
with torch.no_grad():
    outputs = model(**inputs)
    logits = outputs.logits
    predictions = torch.argmax(logits, dim=1)

# 输出结果
for sent, pred in zip(test_sentences, predictions):
    print(f"{sent}")
    print(f"→ Predicted sentiment: {label_map[pred.item()]}\n")


The movie was full of incomprehensibilities.
→ Predicted sentiment: Negative

The movie was full of fun.
→ Predicted sentiment: Positive

The movie was full of excitement.
→ Predicted sentiment: Positive

The movie was full of crap.
→ Predicted sentiment: Negative

The movie was full of rubbish.
→ Predicted sentiment: Negative



89. アーキテクチャの変更#
問題87とは異なるアーキテクチャ（例えば[CLS]トークンを用いるか、各トークンの最大値プーリングを用いるなど）の分類モデルを設計し、事前学習済みモデルを極性分析タスク向けにファインチューニングせよ。検証セット上でファインチューニングされたモデルの正解率を計測せよ。

In [4]:
import torch
import torch.nn as nn
from transformers import BertModel

class BertMaxPoolClassifier(nn.Module):
    def __init__(self, model_name="bert-base-uncased", num_labels=2):
        super().__init__()
        self.bert = BertModel.from_pretrained(model_name)
        self.classifier = nn.Linear(self.bert.config.hidden_size, num_labels)

    def forward(self, input_ids, attention_mask, labels=None):
        outputs = self.bert(
            input_ids=input_ids,
            attention_mask=attention_mask
        )

        # last hidden state: (batch, seq_len, hidden)
        last_hidden = outputs.last_hidden_state

        # max pooling over token dimension
        pooled, _ = torch.max(last_hidden, dim=1)

        logits = self.classifier(pooled)

        loss = None
        if labels is not None:
            loss_fn = nn.CrossEntropyLoss()
            loss = loss_fn(logits, labels)

        return {"loss": loss, "logits": logits}


In [5]:
model = BertMaxPoolClassifier().to(device)
optimizer = AdamW(model.parameters(), lr=2e-5)

num_epochs = 1  # 课程作业足够
model.train()

for epoch in range(num_epochs):
    total_loss = 0
    for batch in train_loader:
        batch = {k: v.to(device) for k, v in batch.items()}
        optimizer.zero_grad()

        outputs = model(
            input_ids=batch["input_ids"],
            attention_mask=batch["attention_mask"],
            labels=batch["labels"]
        )

        loss = outputs["loss"]
        loss.backward()
        optimizer.step()

        total_loss += loss.item()

    print(f"Epoch {epoch+1}, avg loss = {total_loss / len(train_loader):.4f}")


Epoch 1, avg loss = 0.2008


In [6]:
model.eval()
correct = 0
total = 0

with torch.no_grad():
    for batch in dev_loader:
        batch = {k: v.to(device) for k, v in batch.items()}

        outputs = model(
            input_ids=batch["input_ids"],
            attention_mask=batch["attention_mask"]
        )

        preds = torch.argmax(outputs["logits"], dim=1)
        correct += (preds == batch["labels"]).sum().item()
        total += batch["labels"].size(0)

accuracy = correct / total
print("Dev accuracy:", accuracy)


Dev accuracy: 0.9197247706422018
